# Testing YieldFactorModels Filter Functionality

This notebook tests the filter functions in the YieldFactorModels.jl package.

## 1. Setup and Import

In [1]:
# Add the package to the environment
using Pkg
Pkg.activate(".")
Pkg.instantiate()


  Activating project at `~/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl`
Precompiling project...
   2140.6 ms  ✓ YieldFactorModels
  1 dependency successfully precompiled in 5 seconds. 432 already precompiled.


## 1.5 Test Performance Improvements

The model has been optimized based on profiling data:
- ✅ Pre-allocated buffers for network outputs
- ✅ Pre-computed maturity transformations  
- ✅ In-place operations in `update_factor_loadings!`
- ✅ Reduced allocations in hot loops

Expected improvements: 15-30% faster, 50% fewer allocations

In [ ]:
# Import the package and required dependencies
using Revise
using YieldFactorModels
using LinearAlgebra
using ForwardDiff
using Random

# NOTE: `export VAR=...` is a shell command and is not valid Julia syntax in a code cell.
# For runtime settings that can be changed from within Julia use `ENV` or library APIs.
# Set BLAS / native libraries thread knobs where possible:
ENV["OPENBLAS_NUM_THREADS"] = "1"
ENV["OMP_NUM_THREADS"] = "1"
ENV["MKL_NUM_THREADS"] = "1"  # If MKL.jl is used, prefer MKL.set_num_threads(1)
# Also set BLAS threads from Julia (affects LinearAlgebra.BLAS):
LinearAlgebra.BLAS.set_num_threads(1)

# Important: `JULIA_NUM_THREADS` controls Julia's worker threads and must be
# set before the Julia process / kernel is started. You cannot change the number
# of Julia threads from inside a running kernel. To run the kernel with 1 thread,
# start Jupyter / the kernel with the env var set (see instructions below).

Random.seed!(123)  # For reproducibility


TaskLocalRNG()

In [3]:
cd("..")

## 3. Test Individual Functions

### 3.1 Test `initialize_filter!`

In [4]:
println(pwd())

/Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects


In [5]:
data, maturities = load_data("YieldFactorModels.jl/data/", "6")

([7.83403892981863 8.30247299311406 … 4.3268632601382855 4.155682521352292; 7.96778641185185 8.46780405699119 … 4.3328289043004515 4.159368653729067; … ; 10.9519681799619 11.657231239239 … 4.125693391252582 4.537209692896116; 11.371582727334 12.0250709896095 … 4.324326305225288 4.7829716342137685], [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0  …  30.0, 36.0, 48.0, 60.0, 72.0, 84.0, 96.0, 108.0, 120.0, 180.0])

### 3.2 Test `get_β_OLS!`

In [6]:
# Test the OLS estimation function
Z_test = randn(24, 3)
y_test = randn(24)
beta_test = zeros(3)
YieldFactorModels.get_β_OLS!(beta_test, Z_test, y_test)

### 3.4 Test Optimziation


In [ ]:
float_type = Float64
model, model_type = YieldFactorModels.create_model("3SSD-NNS", maturities,24, 3, float_type, "YieldFactorModels.jl/results/thread_id__6/")
param_groups = YieldFactorModels.get_param_groups(model, String[])
all_params = YieldFactorModels.load_initial_parameters!(model, model_type, float_type)
YieldFactorModels.set_params!(model, all_params[:, 1])
# Load static parameters if applicable
all_params[:,1] = YieldFactorModels.load_static_parameters!(model, model_type, "YieldFactorModels.jl/results/", "6", all_params[:,1])
# Convert parameters to appropriate float type
all_params = convert(Matrix{float_type}, all_params)
results = predict(model, data[:, 1:end])


Default param groups assigned.


(preds = [7.105818980299406 7.6411683541625255 … 4.101813892911552 3.816505069422111; 8.307707862166414 8.925321335725757 … 4.100203256309404 3.928089737065757; … ; 10.246774008385044 10.923512835938261 … 3.9080781786568353 4.16701116054946; 12.222354026224087 12.196120154093355 … 4.099407817628048 4.243196429539459], factors = [12.222354026224087 12.196120154093355 … 4.099407817628048 4.243196429539459; -5.116535045924681 -4.55495179993083 … 0.0024060752835034656 -0.4266913601173481; -15.604757157886546 -11.921546769132217 … -0.7472304033888312 -0.3114776724639323], states = [-0.6637548353894249 -0.5166738975593986 … -0.6118330043496237 -1.452318837692889; 0.3953851516476877 0.2437680239464998 … 0.0030209877987692497 0.9110730303332323; … ; -0.36799361338334463 -0.7668893265091452 … 0.1333311665978967 0.106544928535506; -0.033673737495359055 0.056142497094719704 … 0.005807873728042233 0.006333627241489132], factor_loadings_1 = [1.0 1.0 … 1.0 1.0; 0.6801370892287923 0.5380728729989049 

In [8]:
get_loss(model, data)

-4.052454075013805e8

In [9]:
# Alternative: Use BenchmarkTools for detailed timing
using BenchmarkTools

println("Benchmarking with BenchmarkTools...")
println("(This may take a minute...)\n")

benchmark_result = @benchmark YieldFactorModels.get_loss(
    $model, 
    $data, 
) samples=50 evals=3

display(benchmark_result)

println("\n" * "="^60)
println("Summary:")
println("  Minimum time: $(minimum(benchmark_result.times) / 1e9) seconds")
println("  Median time:  $(median(benchmark_result.times) / 1e9) seconds")
println("  Mean time:    $(mean(benchmark_result.times) / 1e9) seconds")
println("  Allocations:  $(benchmark_result.allocs)")
println("  Memory:       $(benchmark_result.memory / 1e6) MB")
println("="^60)

Benchmarking with BenchmarkTools...
(This may take a minute...)


Summary:
  Minimum time: 0.010683333333333335 seconds
  Median time:  0.01129725 seconds
  Mean time:    0.011378703053333335 seconds
  Allocations:  52729
  Memory:       37.67472 MB


BenchmarkTools.Trial: 50 samples with 3 evaluations per sample.
 Range (min … max):  10.683 ms …  14.061 ms  ┊ GC (min … max): 17.03% … 25.15%
 Time  (median):     11.297 ms               ┊ GC (median):    15.80%
 Time  (mean ± σ):   11.379 ms ± 529.118 μs  ┊ GC (mean ± σ):  16.05% ±  2.06%

          ▁▆▁▁ █                                                
  ▇▄▇▇▄▇▇▇████▇█▇▄▄▄▇▁▄▇▁▄▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  10.7 ms         Histogram: frequency by time         14.1 ms <

 Memory estimate: 35.93 MiB, allocs estimate: 52729.

In [10]:
# using Profile, BenchmarkTools

# # Warm up
# YieldFactorModels.get_loss(model, data)

# # Profile with allocation tracking
# Profile.clear()
# Profile.Allocs.clear()

# # Run with allocation profiling
# Profile.Allocs.@profile sample_rate=1.0 begin
#     for i in 1:1
#         YieldFactorModels.get_loss(model, data)
#     end
# end

# # Get results and display
# results = Profile.Allocs.fetch()

# println("\n" * "="^80)
# println("ALLOCATION PROFILING RESULTS")
# println("="^80)

# # Print to stdout with proper IO argument
# Profile.Allocs.print(stdout, results; mincount=10)

# println("\n" * "="^80)
# println("\nAlternative: Show all allocations sorted by size")
# println("="^80)

# # Manual analysis: group and sort by location
# alloc_summary = Dict{String, Int}()
# for alloc in results.allocs
#     location = "$(alloc.type) at $(basename(String(alloc.file))):$(alloc.line)"
#     alloc_summary[location] = get(alloc_summary, location, 0) + 1
# end

# # Sort by count
# sorted_allocs = sort(collect(alloc_summary), by=x->x[2], rev=true)

# println("\nTop allocation sites:")
# for (i, (location, count)) in enumerate(sorted_allocs[1:min(20, end)])
#     println("$i. [$count allocs] $location")
# end

### 3.4 Test Handling Missing Data

In [13]:
# set pwd one level back 

YieldFactorModels.run("6", 231, 12, false, "1SD-NNS", Float64; window_type = "expanding",  max_group_iters=10, run_optimization=true, reestimate=false, group_tol = 1e-6  )
# vcat(fill("1", 22), fill("2", 12) )

Default param groups assigned.
The param groups are : ["1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "2", "2", "2", "2", "2", "2", "2", "2", "2", "2", "2", "2"]
✓ Found valid initial parameters after 0 perturbations

Starting block-coordinate optimization

--- Starting point 1/1 ---
Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0     8.424991e-02     1.109464e-01
 * time: 0.00011301040649414062
    10     8.423313e-02     2.727767e-04
 * time: 0.5833930969238281
    20     8.423313e-02     1.361254e-04
 * time: 0.8954930305480957
    30     8.423313e-02     2.276319e-04
 * time: 1.8571910858154297
    40     8.423313e-02     1.154251e-04
 * time: 2.81882905960083
    50     8.416356e-02     9.596173e-05
 * time: 3.097111940383911
    60     8.412087e-02     7.161194e-05
 * time: 3.322880983352661
    70     8.411155e-02     7.537585e-04
 * time: 4.315988063812256
    80     8.411

MSEDNeuralModel{Float64, Float64, Float64}(MSEDrivenBaseModel{Float64, Float64, Float64}([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0  …  30.0, 36.0, 48.0, 60.0, 72.0, 84.0, 96.0, 108.0, 120.0, 180.0], 24, 3, 18, [1.0 1.0 0.0; 1.0 0.9346831535999188 0.044712759011598345; … ; 1.0 0.0 0.0976613620887947; 1.0 0.0 0.0], [4.636478485536095, -0.46084850559456253, -1.1373094853651702], [0.9887728988586997 -0.03232513270766413 0.013218942090475859; -0.0054261044333230035 0.9500102076301884 0.03776488102022728; 0.00044437169345969016 0.0720661031904868 0.9473972136904782], [0.247235127071713, 0.6257342184532874, -0.09606466313706107], [0.02427254864749525, 0.03624971785186249, -0.050257359995867806], [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2], [0.04920195353812575, 0.011125594052188496, 2.595281957677499, 0.13321376919831476, 0.40400954888976803, 0.3758597245595352, 0.4808709848570262, 2.344342392347071, 2.1980888342446843, 0.22248384424586787, 2.3767709502793917, 0.0826738171

## 4. Test `get_mse` Function

Test the mean squared error calculation over the full dataset.

In [11]:
using CairoMakie
using DelimitedFiles

InterruptException: InterruptException:

In [12]:
model_name = "SD-NS"
# read /Users/siccokooiker/surfdrive/JuliaProjects/YieldFactorModels.jl/results/SD-NS/SD-NS__thread_id__6__factors_filtered_outofsample.csv
filtered_data = readdlm("/Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl/results/$(model_name)/$(model_name)__thread_id__6__factors_filtered_outofsample.csv", ',')
# /Users/siccokooiker/surfdrive/JuliaProjects/YieldFactorModels.jl/data/thread_id__6__data.csv
data = readdlm("/Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl/data/thread_id__6__data.csv", ',')
data = data'
# print shape
println(size(data))
println(size(filtered_data))

# plot first 3 factors over time using CairoMakie

f = Figure(resolution = (900, 500))
ax = Axis(f[1, 1], xlabel = "Time", ylabel = "Value", title = "Factors and Yield Curve Components")
lines!(ax, filtered_data[:, 1], label = "Factor 1")
lines!(ax, filtered_data[:, 2], label = "Factor 2")
lines!(ax, filtered_data[:, 3], label = "Factor 3")

# plot level, slope and curvature from data
# level is last column
lines!(ax, data[1:end-1, end], label = "Level")
# slope is difference between first and last column
lines!(ax, data[1:end-1, 1] .- data[1:end-1, end], label = "Slope")
# curvature: 2 x 14th column - (1st + last)
lines!(ax, 2 .* data[1:end-1, 14] .- (data[1:end-1, 1] .+ data[1:end-1, end]), label = "Curvature")

axislegend(ax, position = :rb)
f

UndefVarError: UndefVarError: `readdlm` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing DelimitedFiles in the current active module Main

In [13]:


fac_path = "/Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl/results/$(model_name)/$(model_name)__thread_id__6__factor_loadings_2_filtered_outofsample.csv"
mat_path = "/Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl/data/thread_id__6__maturities.csv"

Z = readdlm(fac_path, ',', Float64)      # (T × M)
Y = vec(readdlm(mat_path, ',', Float64)) # (M)
T, M = size(Z)
t = 1:T

# --- Create wireframe ---
fig = Figure(resolution = (900, 600))
ax = Axis3(fig[1, 1];
    xlabel="Time",
    ylabel="Maturity",
    zlabel="Loading",
    aspect=(2.0, 1.0, 1.0),  # Make time axis 2× as long as Y-axis
    elevation=25 * π / 180,
    azimuth=45 * π / 180,
    xreversed=true,
    backgroundcolor=:gray20,
    xgridcolor=(:gray60, 0.5),
    ygridcolor=(:gray60, 0.5),
    zgridcolor=(:gray60, 0.5),
    xticklabelsize=10,
    yticklabelsize=10,
    zticklabelsize=10,
    xlabelsize=12,
    ylabelsize=12,
    zlabelsize=12,

)

# draw lines for each maturity
for m in 1:M
    lines!(ax, t, fill(Y[m], T), Z[:, m], color=:cyan)
end
fig

UndefVarError: UndefVarError: `readdlm` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing DelimitedFiles in the current active module Main

In [14]:
# plot predictions /Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl/results/NS/NS__thread_id__6__fit_filtered_outofsample.csv
fit_data = readdlm("/Users/siccokooiker/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/surfdrive/JuliaProjects/YieldFactorModels.jl/results/$(model_name)/$(model_name)__thread_id__6__fit_filtered_outofsample.csv", ',')

# size is (480, 24)
println(size(fit_data))

# plot all 24 lines in one plot 
f2 = Figure(resolution = (900, 500))
ax2 = Axis(f2[1, 1], xlabel = "Maturity", ylabel = "Yield", title = "Yield Curve Fits Over Time")
for i in 1:size(fit_data, 2)
    lines!(ax2, 1:size(fit_data, 1), fit_data[:, i])
end
f2


UndefVarError: UndefVarError: `readdlm` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing DelimitedFiles in the current active module Main

## 5. Test `predict` Function

Test the full prediction over all time periods.

In [15]:
# Test predict function
println("Testing predict function...")
println("This function should return:")
println("  - Factors (M × T matrix)")
println("  - States (L × T matrix)")
println("  - Predictions (N × T matrix)")
println("  All stacked vertically: (M+L+N) × T")

# Note: Call with your actual model
# results = predict(model, data)
# println("\nExpected output shape: ", (M+L+N, T))
# println("Actual output shape: ", size(results))

## 6. Visualization (Optional)

Visualize the predictions vs actual data.

In [ ]:
# Using Plots.jl for visualization
using Plots

# Plot actual yield data
plot(1:T, data[1, :], label="Yield 1 (actual)", 
     xlabel="Time", ylabel="Yield", 
     title="Yield Curve Data", linewidth=2)
plot!(1:T, data[3, :], label="Yield 3 (actual)", linewidth=2)
plot!(1:T, data[5, :], label="Yield 5 (actual)", linewidth=2)

In [ ]:
# After running predict, you can plot predictions vs actuals
# pred_start = M + L + 1
# predictions = results[pred_start:end, :]

# plot(1:T, data[1, :], label="Actual Yield 1", linewidth=2)
# plot!(1:T, predictions[1, :], label="Predicted Yield 1", 
#       linestyle=:dash, linewidth=2)
# title!("Actual vs Predicted Yields")

## 7. Performance and Diagnostics

In [ ]:
# Calculate prediction errors
# residuals = data .- predictions
# mse_per_yield = mean(residuals.^2, dims=2)

# println("MSE per yield:")
# for i in 1:N
#     println("  Yield $i: ", mse_per_yield[i])
# end

In [ ]:
# Plot factor evolution
# factors = results[1:M, :]

# plot(1:T, factors[1, :], label="Factor 1 (Level)", linewidth=2)
# plot!(1:T, factors[2, :], label="Factor 2 (Slope)", linewidth=2)
# plot!(1:T, factors[3, :], label="Factor 3 (Curvature)", linewidth=2)
# title!("Factor Evolution Over Time")
# xlabel!("Time")
# ylabel!("Factor Value")

## 8. Summary

This notebook demonstrates testing of the filter functionality. To fully run it, you'll need to:

1. Ensure your model type implements `AbstractYieldFactorModel`
2. Create a concrete model instance with appropriate parameters
3. Uncomment and run the actual function calls
4. Verify that all functions work correctly with your model structure